In [3]:
import pandas as pd
import glob

folder_path = 'ScenarioData/'

csv_files = glob.glob(folder_path + '/*.csv')

df_list = []


for filename in csv_files:
    df = pd.read_csv(filename)
    df.columns = df.columns.str.strip().str.replace(' ', '')
    df_list.append(df)

combined_df = pd.concat(df_list, ignore_index=True)

combined_csv_path = folder_path + '\\combined.csv'  
combined_df.to_csv(combined_csv_path, index=False)  

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the data
dafni_impacts_path = 'ScenarioData/combined.csv'
oidj_table_processed_path = 'OiDjTable.csv'

dafni_impacts_df = pd.read_csv(dafni_impacts_path)
oidj_table_processed_df = pd.read_csv(oidj_table_processed_path)

# Data Integration
oidj_table_processed_df.rename(columns={'zonei': 'zone'}, inplace=True)
dafni_impacts_enriched_df = pd.merge(dafni_impacts_df, oidj_table_processed_df, left_on='net_i', right_on='zone', how='left')
dafni_impacts_enriched_df.rename(columns={'Oi_all': 'Oi_net_i', 'Dj_all': 'Dj_net_i'}, inplace=True)
dafni_impacts_enriched_df.drop(['zone'], axis=1, inplace=True)

dafni_impacts_enriched_df = pd.merge(dafni_impacts_enriched_df, oidj_table_processed_df, left_on='net_j', right_on='zone', how='left')
dafni_impacts_enriched_df.rename(columns={'Oi_all': 'Oi_net_j', 'Dj_all': 'Dj_net_j'}, inplace=True)
dafni_impacts_enriched_df.drop(['zone'], axis=1, inplace=True)


dafni_impacts_enriched_df.to_csv('OiDj_merged.csv')

: 